# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846824039322                   -0.70    4.5         
  2   -7.852317898267       -2.26       -1.53    1.0   26.5ms
  3   -7.852616133296       -3.53       -2.56    1.5   29.0ms
  4   -7.852645955639       -4.53       -2.88    2.5   36.0ms
  5   -7.852646493580       -6.27       -3.16    1.2   28.2ms
  6   -7.852646679082       -6.73       -4.15    1.0   26.6ms
  7   -7.852646686665       -8.12       -4.94    2.2   35.7ms
  8   -7.852646686723      -10.24       -5.43    1.2   28.7ms
  9   -7.852646686727      -11.31       -5.57    1.2   28.2ms
 10   -7.852646686730      -11.63       -6.67    1.0   27.3ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846859614525                   -0.70           4.8         
  2   -7.852524565648       -2.25       -1.64   0.80    2.0    269ms
  3   -7.852635689086       -3.95       -2.72   0.80    1.0   24.5ms
  4   -7.852646410160       -4.97       -3.24   0.80    2.2   32.0ms
  5   -7.852646662176       -6.60       -4.05   0.80    1.0   24.0ms
  6   -7.852646686384       -7.62       -4.66   0.80    2.0   31.8ms
  7   -7.852646686713       -9.48       -5.37   0.80    1.2   26.5ms
  8   -7.852646686729      -10.81       -6.53   0.80    1.8   29.0ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.434418e+01     3.147616e+00
 * time: 0.06523418426513672
     1     1.329257e+00     1.617905e+00
 * time: 0.29673194885253906
     2    -1.573576e+00     2.001335e+00
 * time: 0.3219790458679199
     3    -3.696988e+00     1.645962e+00
 * time: 0.35787510871887207
     4    -5.025247e+00     1.641009e+00
 * time: 0.39331912994384766
     5    -6.775005e+00     9.193419e-01
 * time: 0.42879700660705566
     6    -7.506676e+00     3.877142e-01
 * time: 0.46477198600769043
     7    -7.733731e+00     2.182918e-01
 * time: 0.49025416374206543
     8    -7.796164e+00     1.045124e-01
 * time: 0.5154900550842285
     9    -7.821712e+00     8.950826e-02
 * time: 0.541046142578125
    10    -7.832699e+00     5.926363e-02
 * time: 0.5662620067596436
    11    -7.838898e+00     4.605177e-02
 * time: 0.5914731025695801
    12    -7.843352e+00     4.241325e-02
 * time: 0.616569995880127
    13    -7.844948e+00     5.806671e-02
 * time: 0.64152

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846857983890                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645903080                   -1.64         
  2   -7.852646686730       -6.11       -3.70    2.06s
  3   -7.852646686730      -13.30       -7.24    161ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.31858171615231e-7
|ρ_newton - ρ_scfv| = 8.016654374519267e-7
|ρ_newton - ρ_dm|   = 5.93950299875608e-10
